<a href="https://colab.research.google.com/github/RAvila-bioeng/M.R.AI/blob/main/alzheimer's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle_API"

# Descargar dataset
!kaggle datasets download -d ninadaithal/imagesoasis

# Descomprimir
!unzip -q imagesoasis.zip

import os
print("Contenido del directorio actual:", os.listdir("."))
print("Contenido de 'Data' si existe:", os.listdir("Data") if os.path.exists("Data") else "No existe 'Data'")


Mounted at /content/gdrive
Dataset URL: https://www.kaggle.com/datasets/ninadaithal/imagesoasis
License(s): apache-2.0
 99% 1.22G/1.23G [00:05<00:00, 239MB/s]
100% 1.23G/1.23G [00:06<00:00, 218MB/s]
Contenido del directorio actual: ['.config', 'gdrive', 'Data', 'imagesoasis.zip', 'sample_data']
Contenido de 'Data' si existe: ['Moderate Dementia', 'Non Demented', 'Mild Dementia', 'Very mild Dementia']


In [ ]:
import os
import shutil
import random
from collections import defaultdict

original_dir = "./Data"
assert os.path.exists(original_dir), "No existe ./Data. Revisa la celda 1."

subset_dir = "./Data_subset"

# Empezar *limpio*
if os.path.exists(subset_dir):
    shutil.rmtree(subset_dir)
os.makedirs(subset_dir, exist_ok=True)

valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
N_PATIENTS_NON_DEMENTED = 120  # nº máximo de pacientes sanos

classes = [d for d in os.listdir(original_dir)
           if os.path.isdir(os.path.join(original_dir, d))]
print("Clases encontradas en Data:", classes)

for cls in classes:
    src_cls_dir = os.path.join(original_dir, cls)
    dst_cls_dir = os.path.join(subset_dir, cls)
    os.makedirs(dst_cls_dir, exist_ok=True)

    print(f"\nProcesando clase: {cls}")

    if "non" not in cls.lower():   # todas menos Non Demented
        imgs = [f for f in os.listdir(src_cls_dir)
                if f.lower().endswith(valid_exts)]
        print(f"  Copiando TODAS las imágenes ({len(imgs)})...")
        for fname in imgs:
            shutil.copy2(os.path.join(src_cls_dir, fname),
                         os.path.join(dst_cls_dir, fname))

    else:
        # Non Demented: reducir por nº de pacientes
        imgs = [f for f in os.listdir(src_cls_dir)
                if f.lower().endswith(valid_exts)]
        print(f"  Imágenes totales en {cls}: {len(imgs)}")
        print("  Ejemplos de nombres:", imgs[:5])

        images_by_patient = defaultdict(list)
        for fname in imgs:
            parts = fname.split('_')
            # OAS1_0097_MR1_mpr-3_127.jpg → paciente = "OAS1_0097"
            patient_id = "_".join(parts[:2]) if len(parts) >= 2 else parts[0]
            images_by_patient[patient_id].append(fname)

        print(f"  Pacientes totales detectados: {len(images_by_patient)}")

        patients = list(images_by_patient.keys())
        n_keep = min(N_PATIENTS_NON_DEMENTED, len(patients))
        selected_patients = random.sample(patients, n_keep)

        print(f"  Pacientes que vamos a conservar en {cls}: {n_keep}")

        count_imgs = 0
        for pid in selected_patients:
            for fname in images_by_patient[pid]:
                shutil.copy2(os.path.join(src_cls_dir, fname),
                             os.path.join(dst_cls_dir, fname))
                count_imgs += 1

        print(f"  Imágenes copiadas en {cls} (subset): {count_imgs}")

print("\n✅ Data_subset creado en:", subset_dir)

# Comprobar conteos
for cls in os.listdir(subset_dir):
    cls_path = os.path.join(subset_dir, cls)
    if os.path.isdir(cls_path):
        n = len([f for f in os.listdir(cls_path)
                 if f.lower().endswith(valid_exts)])
        print(f"{cls}: {n} imágenes en Data_subset")


Clases encontradas en Data: ['Moderate Dementia', 'Non Demented', 'Mild Dementia', 'Very mild Dementia']

Procesando clase: Moderate Dementia
  Copiando TODAS las imágenes (488)...

Procesando clase: Non Demented
  Imágenes totales en Non Demented: 67222
  Ejemplos de nombres: ['OAS1_0097_MR1_mpr-3_127.jpg', 'OAS1_0136_MR1_mpr-4_144.jpg', 'OAS1_0317_MR1_mpr-2_155.jpg', 'OAS1_0212_MR1_mpr-2_106.jpg', 'OAS1_0162_MR1_mpr-1_128.jpg']
  Pacientes totales detectados: 266
  Pacientes que vamos a conservar en Non Demented: 120
  Imágenes copiadas en Non Demented (subset): 30500

Procesando clase: Mild Dementia
  Copiando TODAS las imágenes (5002)...

Procesando clase: Very mild Dementia
  Copiando TODAS las imágenes (13725)...

✅ Data_subset creado en: ./Data_subset
Moderate Dementia: 488 imágenes en Data_subset
Non Demented: 30500 imágenes en Data_subset
Mild Dementia: 5002 imágenes en Data_subset
Very mild Dementia: 13725 imágenes en Data_subset


In [ ]:
import os
import shutil
import random
from collections import defaultdict

subset_dir = "./Data_subset"          # de aquí leemos
split_root = "./Data_split_3cls"      # aquí escribimos el nuevo split

# Limpiar split anterior (si existe)
if os.path.exists(split_root):
    shutil.rmtree(split_root)
os.makedirs(split_root, exist_ok=True)

splits = ["train", "val", "test"]
for s in splits:
    os.makedirs(os.path.join(split_root, s), exist_ok=True)

valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')

# Mapeo de clases físicas -> clases lógicas
# Moderate Dementia se fusiona con Mild Dementia
CLASS_MAP = {
    "Non Demented": "Non Demented",
    "Very mild Dementia": "Very mild Dementia",
    "Mild Dementia": "Mild Dementia",
    "Moderate Dementia": "Mild Dementia",
}

# 1) Construimos un diccionario: logical_class -> patient_id -> lista de rutas de imagen
class_patient_images = defaultdict(lambda: defaultdict(list))

physical_classes = [d for d in os.listdir(subset_dir)
                    if os.path.isdir(os.path.join(subset_dir, d))]
print("Clases físicas en subset:", physical_classes)

for phys_cls in physical_classes:
    if phys_cls not in CLASS_MAP:
        print(f"  Aviso: clase {phys_cls} no está en CLASS_MAP, se ignora.")
        continue

    logical_cls = CLASS_MAP[phys_cls]
    cls_dir = os.path.join(subset_dir, phys_cls)

    print(f"\nLeyendo clase física '{phys_cls}' como clase lógica '{logical_cls}'")

    for fname in os.listdir(cls_dir):
        if not fname.lower().endswith(valid_exts):
            continue

        # Sacar patient_id de nombres tipo: OAS1_0028_MR1_mpr-1_100.jpg
        parts = fname.split('_')
        patient_id = "_".join(parts[:2]) if len(parts) >= 2 else parts[0]

        full_path = os.path.join(cls_dir, fname)
        class_patient_images[logical_cls][patient_id].append(full_path)

# 2) Hacemos el split por paciente para cada clase lógica
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

for logical_cls, patients_dict in class_patient_images.items():
    print(f"\n=== Splitting logical class: {logical_cls} ===")

    patient_ids = list(patients_dict.keys())
    random.shuffle(patient_ids)
    n_total = len(patient_ids)

    # nº de pacientes por split
    n_train = max(1, int(TRAIN_RATIO * n_total))
    n_val = max(1, int(VAL_RATIO * n_total))
    n_test = n_total - n_train - n_val
    if n_test <= 0:
        n_test = 1
        n_train = max(1, n_train - 1)

    train_patients = patient_ids[:n_train]
    val_patients = patient_ids[n_train:n_train + n_val]
    test_patients = patient_ids[n_train + n_val:]

    print(f"  Pacientes: total={n_total}, train={len(train_patients)}, val={len(val_patients)}, test={len(test_patients)}")

    # Crear carpetas de clase lógica en cada split
    for s in splits:
        os.makedirs(os.path.join(split_root, s, logical_cls), exist_ok=True)

    def copy_group(pat_list, split_name):
        dst_base = os.path.join(split_root, split_name, logical_cls)
        count = 0
        for pid in pat_list:
            for src_path in patients_dict[pid]:
                fname = os.path.basename(src_path)
                dst_path = os.path.join(dst_base, fname)

                # Si por alguna razón ya existe ese nombre, renombra para evitar sobrescribir
                if os.path.exists(dst_path):
                    name, ext = os.path.splitext(fname)
                    dst_path = os.path.join(dst_base, f"{name}_dup{ext}")

                shutil.copy2(src_path, dst_path)
                count += 1
        print(f"    {split_name}: {count} imágenes")

    copy_group(train_patients, "train")
    copy_group(val_patients, "val")
    copy_group(test_patients, "test")

print("\n✅ Split de 3 clases creado en:", split_root)


Clases físicas en subset: ['Moderate Dementia', 'Non Demented', 'Mild Dementia', 'Very mild Dementia']

Leyendo clase física 'Moderate Dementia' como clase lógica 'Mild Dementia'

Leyendo clase física 'Non Demented' como clase lógica 'Non Demented'

Leyendo clase física 'Mild Dementia' como clase lógica 'Mild Dementia'

Leyendo clase física 'Very mild Dementia' como clase lógica 'Very mild Dementia'

=== Splitting logical class: Mild Dementia ===
  Pacientes: total=23, train=16, val=3, test=4
    train: 3843 imágenes
    val: 732 imágenes
    test: 915 imágenes

=== Splitting logical class: Non Demented ===
  Pacientes: total=120, train=84, val=18, test=18
    train: 21533 imágenes
    val: 4697 imágenes
    test: 4270 imágenes

=== Splitting logical class: Very mild Dementia ===
  Pacientes: total=58, train=40, val=8, test=10
    train: 9638 imágenes
    val: 1891 imágenes
    test: 2196 imágenes

✅ Split de 3 clases creado en: ./Data_split_3cls


In [ ]:
for split in ["train", "val", "test"]:
    print(f"\n=== {split.upper()} ===")
    split_dir = os.path.join(split_root, split)
    for cls in os.listdir(split_dir):
        cls_path = os.path.join(split_dir, cls)
        if os.path.isdir(cls_path):
            n = len([f for f in os.listdir(cls_path)
                     if f.lower().endswith(valid_exts)])
            print(f"  {cls}: {n} imágenes")



=== TRAIN ===
  Non Demented: 21533 imágenes
  Mild Dementia: 3843 imágenes
  Very mild Dementia: 9638 imágenes

=== VAL ===
  Non Demented: 4697 imágenes
  Mild Dementia: 732 imágenes
  Very mild Dementia: 1891 imágenes

=== TEST ===
  Non Demented: 4270 imágenes
  Mild Dementia: 915 imágenes
  Very mild Dementia: 2196 imágenes
